In [2]:
import json

import dgl
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pandas as pd
import torch

DATE_FORMAT = '%m/%d/%Y'

In [3]:
model_name = 'model-05-bce'

In [4]:
G = dgl.load_graphs('dgl_icews_graph.bin')[0][0]

In [5]:
icews = pd.read_csv('icews_redux.csv', header=0, index_col=['Event ID'], parse_dates=['Event Date'], date_format=DATE_FORMAT)
icews.drop(columns=['CAMEO Code', 'Story ID', 'Sentence Number', 'Latitude', 'Longitude'], inplace=True)
icews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 950359 entries, 36615529 to 37764068
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Event Date      950359 non-null  object 
 1   Source Name     950359 non-null  object 
 2   Source Sectors  686161 non-null  object 
 3   Source Country  909074 non-null  object 
 4   Source Type     950359 non-null  object 
 5   Event Text      950359 non-null  object 
 6   Intensity       950359 non-null  float64
 7   Target Name     950359 non-null  object 
 8   Target Sectors  577849 non-null  object 
 9   Target Country  907127 non-null  object 
 10  Target Type     950359 non-null  object 
 11  Publisher       950359 non-null  object 
 12  City            643528 non-null  object 
 13  District        151008 non-null  object 
 14  Province        711140 non-null  object 
 15  Country         950285 non-null  object 
dtypes: float64(1), object(15)
memory usage: 123.3+ MB


In [6]:
entity_countries = icews.groupby('Source Name')['Source Country'].nunique()

In [7]:
entity_countries[entity_countries != 1]

Source Name
Abbott Laboratories                0
Abdullatif Al-Zayani               0
Abdullatif bin Rashid Al-Zayani    0
Achim Steiner                      0
Activist (Greenpeace)              0
                                  ..
World Vision                       0
World Wildlife Fund                0
Wynn Resorts                       0
Zawahiri                           0
Zoran Zaev                         2
Name: Source Country, Length: 386, dtype: int64

In [8]:
ukraine_source = icews[(icews['Source Name'].str.contains('Ukraine')) | (icews['Source Country'] == 'Ukraine')]
ukraine_source.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53523 entries, 36615651 to 37763972
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Event Date      53523 non-null  object 
 1   Source Name     53523 non-null  object 
 2   Source Sectors  37027 non-null  object 
 3   Source Country  53523 non-null  object 
 4   Source Type     53523 non-null  object 
 5   Event Text      53523 non-null  object 
 6   Intensity       53523 non-null  float64
 7   Target Name     53523 non-null  object 
 8   Target Sectors  28091 non-null  object 
 9   Target Country  50379 non-null  object 
 10  Target Type     53523 non-null  object 
 11  Publisher       53523 non-null  object 
 12  City            34199 non-null  object 
 13  District        2864 non-null   object 
 14  Province        35604 non-null  object 
 15  Country         53523 non-null  object 
dtypes: float64(1), object(15)
memory usage: 6.9+ MB


In [9]:
ukraine_source.head()

,Event Date,Source Name,Source Sectors,Source Country,Source Type,Event Text,Intensity,Target Name,Target Sectors,Target Country,Target Type,Publisher,City,District,Province,Country
Event ID,,,,,,,,,,,,,,,,
36615651,2022-01-01,Reserve Personnel (Ukraine),"Government,Military",Ukraine,Other,"Arrest, detain, or charge with legal action",-5.0,Citizen (Ukraine),"General Population / Civilian / Social,Social",Ukraine,Organization,Unknown,Lviv,NaN,L'vivs'ka Oblast',Ukraine
36615720,2022-01-01,Volodymyr Zelensky,"Elite,Executive,Executive Office,Government",Ukraine,Name,Express intent to meet or negotiate,4.0,Joseph Robinette Biden,"(National) Major Party,Cabinet,Center Left,Exe...",United States,Name,BBC Monitoring Ukraine & Baltics,NaN,NaN,NaN,Ukraine
36615796,2022-01-01,Media Personnel (Ukraine),"Media,Social",Ukraine,Organization,Make statement,0.0,Russia,NaN,Russian Federation,Country,BBC Monitoring Ukraine & Baltics,NaN,NaN,NaN,Ukraine
36615832,2022-01-01,Ukraine,NaN,Ukraine,Country,Consult,1.0,United States,NaN,United States,Country,Unknown,NaN,NaN,NaN,Ukraine
36615840,2022-01-01,Volodymyr Zelensky,"Elite,Executive,Executive Office,Government",Ukraine,Name,"Impose embargo, boycott, or sanctions",-8.0,Company - Owner or Operator (Ukraine),"Business,Social",Ukraine,Other,BBC Monitoring Ukraine & Baltics,NaN,NaN,NaN,Ukraine


In [10]:
ukraine_target = icews[(icews['Target Name'].str.contains('Ukraine')) | (icews['Target Country'] == 'Ukraine')]
ukraine_target.info()

<class 'pandas.core.frame.DataFrame'>
Index: 82538 entries, 36615573 to 37764012
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Event Date      82538 non-null  object 
 1   Source Name     82538 non-null  object 
 2   Source Sectors  50363 non-null  object 
 3   Source Country  78572 non-null  object 
 4   Source Type     82538 non-null  object 
 5   Event Text      82538 non-null  object 
 6   Intensity       82538 non-null  float64
 7   Target Name     82538 non-null  object 
 8   Target Sectors  27319 non-null  object 
 9   Target Country  82538 non-null  object 
 10  Target Type     82538 non-null  object 
 11  Publisher       82538 non-null  object 
 12  City            53446 non-null  object 
 13  District        6057 non-null   object 
 14  Province        56285 non-null  object 
 15  Country         82538 non-null  object 
dtypes: float64(1), object(15)
memory usage: 10.7+ MB


In [11]:
ukraine_target.head()

,Event Date,Source Name,Source Sectors,Source Country,Source Type,Event Text,Intensity,Target Name,Target Sectors,Target Country,Target Type,Publisher,City,District,Province,Country
Event ID,,,,,,,,,,,,,,,,
36615573,2022-01-01,United States,NaN,United States,Country,Praise or endorse,3.4,Ukraine,NaN,Ukraine,Country,The New York Times,Kiev,NaN,Misto Kyyiv,Ukraine
36615574,2022-01-01,North Atlantic Treaty Organization,"Global,Global Defense / Security IGOs,Internat...",NaN,Organization,Praise or endorse,3.4,Ukraine,NaN,Ukraine,Country,The New York Times,Kiev,NaN,Misto Kyyiv,Ukraine
36615576,2022-01-01,Russia,NaN,Russian Federation,Country,Threaten,-4.4,Ukraine,NaN,Ukraine,Country,The New York Times,Moscow Kremlin,NaN,Moskovskaya Oblast',Russian Federation
36615591,2022-01-01,Russia,NaN,Russian Federation,Country,Use conventional military force,-10.0,Ukraine,NaN,Ukraine,Country,Unknown,NaN,NaN,Washington D.C.,United States
36615592,2022-01-01,Russia,NaN,Russian Federation,Country,Use conventional military force,-10.0,Ukraine,NaN,Ukraine,Country,Unknown,NaN,NaN,Washington D.C.,United States


In [12]:
ukraine_source_event = {}
for i in ukraine_source['Event Text']:
    if i not in ukraine_source_event:
        ukraine_source_event[i] = 0
    ukraine_source_event[i] += 1
print(ukraine_source_event)

{'Arrest, detain, or charge with legal action': 1209, 'Express intent to meet or negotiate': 1739, 'Make statement': 9344, 'Consult': 4756, 'Impose embargo, boycott, or sanctions': 136, 'Sign formal agreement': 665, 'Use unconventional violence': 999, 'Impose state of emergency or martial law': 10, 'Criticize or denounce': 1078, 'Accuse': 2312, 'Make pessimistic comment': 141, 'fight with small arms and light weapons': 3236, 'Discuss by telephone': 557, 'Engage in negotiation': 1350, 'Sexually assault': 12, 'Praise or endorse': 2051, 'Make a visit': 1778, 'Engage in symbolic act': 198, 'Use conventional military force': 4659, 'Engage in diplomatic cooperation': 396, 'Host a visit': 5152, 'Investigate': 238, 'Express intent to engage in diplomatic cooperation (such as policy support)': 384, 'Make an appeal or request': 3051, 'Express intent to cooperate': 429, 'Expel or deport individuals': 72, 'Provide economic aid': 27, 'Make optimistic comment': 315, 'Confiscate property': 97, 'Deman

In [13]:
ukraine_target_event = {}
for i in ukraine_target['Event Text']:
    if i not in ukraine_target_event:
        ukraine_target_event[i] = 0
    ukraine_target_event[i] += 1
print(ukraine_target_event)

{'Praise or endorse': 3572, 'Threaten': 370, 'Use conventional military force': 13689, 'Arrest, detain, or charge with legal action': 1223, 'Express intent to meet or negotiate': 2786, 'Mobilize or increase armed forces': 1716, 'Express intent to engage in diplomatic cooperation (such as policy support)': 1503, 'Consult': 4756, 'Impose embargo, boycott, or sanctions': 63, 'Sign formal agreement': 662, 'Criticize or denounce': 801, 'Use unconventional violence': 1276, 'Impose state of emergency or martial law': 130, 'fight with artillery and tanks': 2393, 'Make statement': 10717, 'Make pessimistic comment': 306, 'fight with small arms and light weapons': 3508, 'Discuss by telephone': 557, 'Engage in diplomatic cooperation': 452, 'Accuse': 1880, 'Engage in negotiation': 1350, 'Express intent to cooperate': 535, 'Provide military aid': 2129, 'Host a visit': 1780, 'Engage in symbolic act': 220, 'Express intent to provide military aid': 1593, 'Make a visit': 5152, 'Investigate': 308, 'Consi

In [16]:
print('Impose curfew', ukraine_source_event['Impose curfew'])
print('Receive inspectors', ukraine_source_event['Receive inspectors'])
print('Appeal for rights', ukraine_source_event['Appeal for rights'])
print('Demand release of persons or property', ukraine_source_event['Demand release of persons or property'])
print('Demand humanitarian aid', ukraine_source_event['Demand humanitarian aid'])
print('Demand settling of dispute', ukraine_source_event['Demand settling of dispute'])
print('Reject request for change in leadership', ukraine_source_event['Reject request for change in leadership'])
print('Obstruct passage, block', ukraine_source_event['Obstruct passage, block'])

Impose curfew 9
Receive inspectors 2
Appeal for rights 2
Demand release of persons or property 1
Demand humanitarian aid 2
Demand settling of dispute 2
Reject request for change in leadership 1
Obstruct passage, block 7


In [17]:
print('Impose curfew', ukraine_target_event['Impose curfew'])
print('Receive inspectors', ukraine_target_event['Receive inspectors'])
print('Appeal for rights', ukraine_target_event['Appeal for rights'])
print('Demand release of persons or property', ukraine_target_event['Demand release of persons or property'])
print('Demand humanitarian aid', ukraine_target_event['Demand humanitarian aid'])
print('Demand settling of dispute', ukraine_target_event['Demand settling of dispute'])
print('Obstruct passage, block', ukraine_target_event['Obstruct passage, block'])

Impose curfew 9
Receive inspectors 1
Appeal for rights 1
Demand release of persons or property 1
Demand humanitarian aid 2
Demand settling of dispute 12
Obstruct passage, block 7
